# 환경 설정
Anaconda Prompt 에서 다음 명령 수행
> pip install opencv-python

In [ ]:
import cv2
cv2.__version__

## 구글 마운트
- 원하는 경로에 설정하세요

# OpenCV (Computer Vision)
다양한 영상 (이미지) / 동영상 처리에 사용되는 오픈소스 라이브러리



-
Google Colab에서는 이미지를 화면에 출력할 때 cv2.imshow()를 사용할 수 없습니다. 이는 Google Colab이 브라우저 기반의 환경으로, cv2.imshow()가 사용하려는 데스크톱 GUI 창을 지원하지 않기 때문입니다. 대신 Colab에서는 이미지를 시각화하기 위해 Jupyter Notebook에 직접 이미지를 출력해야 합니다.

- cv2_imshow는 이를 해결하기 위해 Colab에서 이미지를 표시할 수 있도록 제공되는 함수입니다. from google.colab.patches import cv2_imshow 을 사용하여 Colab의 제한된 GUI 환경에서도 OpenCV 이미지 작업을 원활히 수행할 수 있습니다.



# 1. 이미지 출력

예제 이미지 : https://pixabay.com/images/id-2083492/  
크기 : 640 x 390  
파일명 : img.jpg

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread('./img.jpg') # 해당 경로의 파일 읽어오기
cv2_imshow(img)
# cv2.imshow('img', img) # img 라는 이름의 창에 img 를 표시
# cv2.waitKey(0) # 지정된 시간(ms) 동안 사용자 키 입력 대기
# cv2.destroyAllWindows() # 모든 창 닫기

## 읽기 옵션
1. cv2.IMREAD_COLOR : 컬러 이미지. 투명 영역은 무시 (기본값)
1. cv2.IMREAD_GRAYSCALE : 흑백 이미지
1. cv2.IMREAD_UNCHANGED : 투명 영역까지 포함

In [ ]:
import cv2
img_color = cv2.imread('./img.jpg', cv2.IMREAD_COLOR)
img_gray = cv2.imread('./img.jpg', cv2.IMREAD_GRAYSCALE)
img_unchanged = cv2.imread('./img.jpg', cv2.IMREAD_UNCHANGED)

cv2_imshow(img_color)
cv2_imshow(img_gray)
cv2_imshow(img_unchanged)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

## Shape
이미지의 height, width, channel 정보

In [ ]:
import cv2
img = cv2.imread('./img.jpg')
img.shape # 세로, 가로, Channel

# 2. 동영상 출력

## 동영상 파일 출력

- 예제 동영상 : https://www.pexels.com/video/7515833/
- 크기 : SD (360 x 640)
- 파일명 : video.mp4

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture('./video.mp4')

while cap.isOpened(): # 동영상 파일이 올바로 열렸는지?
    ret, frame = cap.read() # ret : 성공 여부,  frame : 받아온 이미지 (프레임)
    if not ret:
        print('더 이상 가져올 프레임이 없어요')
        break

    cv2_imshow(frame)  # 모든 프레임이 다 출력되므로 굉장히 느리다.

    # if cv2.waitKey(1) == ord('q'):
    #     print('사용자 입력에 의해 종료합니다')
    #     break

cap.release() # 자원 해제
# cv2.destroyAllWindows() # 모든 창 닫기

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# 1) 재생할 동영상 파일 경로 지정
video_path = 'video.mp4'  # Colab 작업 디렉토리에 업로드된 파일명

# 2) 파일을 바이너리로 읽어 Base64로 인코딩
with open(video_path, 'rb') as f:
    video_bytes = f.read()
data_url = "data:video/mp4;base64," + b64encode(video_bytes).decode()

# 3) HTML5 <video> 태그로 삽입
HTML(f"""
<video width="640" height="360" controls autoplay loop>
  <source src="{data_url}" type="video/mp4">
  Your browser does not support HTML5 video.
</video>
""")

## 카메라 출력
- 코랩환경에서는 아래의 코드가 에러 난다.

In [ ]:
import cv2
cap = cv2.VideoCapture(0) # 0번째 카메라 장치 (Device ID)

if not cap.isOpened(): # 카메라가 잘 열리지 않은 경우
    exit() # 프로그램 종료

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2_imshow(frame)
    if cv2.waitKey(1) == ord('q'): # 사용자가 q 를 입력하면
        break

cap.release()
# cv2.destroyAllWindows()

코랩에서는 실시간 웹캠을 적용할 수 없으므로, 영상으로 사진을 촬영하고 프레임을 붙여서 영상을 만들겠습니다. 저장된 영상을 나중에 모델에 적용하는 과정입니다.

- 먼저 영상을 촬영하고 영상을 촬영하여 각 프레임을 저장합니다

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
import cv2
import numpy as np
import PIL.Image
import io
import base64

# JavaScript를 사용해 웹캠 스트림 캡처
def video_stream():
    js = Javascript('''
        async function videoStream() {
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const context = canvas.getContext('2d');

            document.body.appendChild(video);
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            const stopButton = document.createElement('button');
            stopButton.textContent = 'Stop Stream';
            document.body.appendChild(stopButton);

            let stop = false;
            stopButton.onclick = () => stop = true;

            const frames = [];
            while (!stop) {
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                const data = canvas.toDataURL('image/jpeg', 0.8);
                frames.push(data);
                await new Promise(resolve => setTimeout(resolve, 100));  // 100ms 간격
            }

            video.remove();
            stopButton.remove();
            stream.getTracks().forEach(track => track.stop());
            return frames;
        }
    ''')
    display(js)
    return eval_js('videoStream()')

# 캡처된 프레임 데이터를 OpenCV 이미지로 변환
def process_frame(frame_data):
    binary = io.BytesIO(base64.b64decode(frame_data.split(',')[1]))
    img = PIL.Image.open(binary)
    return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

# 프레임 저장
def save_frames(frames):
    for idx, frame_data in enumerate(frames):
        img = process_frame(frame_data)
        filename = f'frame_{idx:04d}.jpg'
        cv2.imwrite(filename, img)
        print(f"Saved {filename}")

# 스트림 시작 및 프레임 저장
print("Starting video stream. Click 'Stop Stream' to end.")
frames = video_stream()
save_frames(frames)
print("All frames have been saved.")


- 저장된 프레임을 모두 모아 하나의 영상으로 만듭니다.

In [ ]:
import cv2
import os

def create_video_from_images(image_folder, output_video, fps=10):
    """
    저장된 이미지를 동영상으로 변환하는 함수.

    Args:
    - image_folder: 이미지가 저장된 폴더 경로.
    - output_video: 생성할 동영상 파일 이름.
    - fps: 초당 프레임 수 (기본값 10).

    Returns:
    - None
    """
    # 폴더 내 이미지 파일 리스트 가져오기
    images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
    images.sort()  # 이미지 파일 이름 순서대로 정렬

    if not images:
        print("No images found in the folder.")
        return

    # 첫 번째 이미지로 프레임 크기 결정
    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape
    size = (width, height)

    # 동영상 작성기 초기화
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # .mp4 파일 포맷
    video = cv2.VideoWriter(output_video, fourcc, fps, size)

    for image in images:
        image_path = os.path.join(image_folder, image)
        frame = cv2.imread(image_path)
        video.write(frame)  # 동영상에 프레임 추가

    video.release()
    print(f"Video saved as {output_video}")

# 사용 예시
image_folder = "./"  # 이미지가 저장된 폴더 경로 (현재 디렉토리)
output_video = image_folder + "output_video.mp4"  # 생성할 동영상 파일 이름
fps = 10  # 초당 프레임 수

create_video_from_images(image_folder, output_video, fps)
